# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [ ]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

In [7]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3
import urllib
import zipfile
import os

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [4]:
#download the data for classification
urllib.request.urlretrieve("https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip", "data/dogImages.zip")

('data/dogImages.zip', <http.client.HTTPMessage at 0x7fc736c87ed0>)

In [6]:
import zipfile
with zipfile.ZipFile("data/dogImages.zip", 'r') as zip_ref:
    zip_ref.extractall("data")


In [11]:
del zip_ref

The dataset contains a selection of dog picures divided into 133 breeds. We can see their distribution in the graph below

In [13]:
dog_breeds = os.listdir('data/dogImages/train')
os.listdir('data/dogImages/train')

['114.Otterhound',
 '110.Norwegian_lundehund',
 '127.Silky_terrier',
 '091.Japanese_chin',
 '095.Kuvasz',
 '061.English_cocker_spaniel',
 '043.Canaan_dog',
 '131.Wirehaired_pointing_griffon',
 '124.Poodle',
 '087.Irish_terrier',
 '053.Cocker_spaniel',
 '064.English_toy_spaniel',
 '105.Neapolitan_mastiff',
 '032.Boston_terrier',
 '017.Bearded_collie',
 '094.Komondor',
 '003.Airedale_terrier',
 '084.Icelandic_sheepdog',
 '122.Pointer',
 '098.Leonberger',
 '125.Portuguese_water_dog',
 '009.American_water_spaniel',
 '058.Dandie_dinmont_terrier',
 '115.Papillon',
 '006.American_eskimo_dog',
 '041.Bullmastiff',
 '085.Irish_red_and_white_setter',
 '081.Greyhound',
 '004.Akita',
 '103.Mastiff',
 '044.Cane_corso',
 '118.Pembroke_welsh_corgi',
 '013.Australian_terrier',
 '117.Pekingese',
 '002.Afghan_hound',
 '025.Black_and_tan_coonhound',
 '129.Tibetan_mastiff',
 '093.Kerry_blue_terrier',
 '023.Bernese_mountain_dog',
 '070.German_pinscher',
 '052.Clumber_spaniel',
 '048.Chihuahua',
 '088.Irish_

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [ ]:
#TODO: Declare your HP ranges, metrics etc.

In [ ]:
#TODO: Create estimators for your HPs

estimator = # TODO: Your estimator here

tuner = # TODO: Your HP tuner here

In [ ]:
# TODO: Fit your HP Tuner
tuner.fit() # TODO: Remember to include your data channels

In [ ]:
# TODO: Get the best estimators and the best HPs

best_estimator = #TODO

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

estimator = # TODO: Your estimator here

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()